# Métrica 1.2.01: "Congruencia entre fechas de obtención del documento probatorio entre los últimos niveles educativos"

In [1]:
import re
import urllib.parse
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure, OperationFailure
import unicodedata
import traceback
from datetime import datetime
# Asumimos que tienes un archivo config.py con tus variables de conexión
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

# --- 1. CONFIGURACIÓN BÁSICA ---
METRIC_ID = "1_2_01_FECHAS_ESCOLARIDAD"
SOURCE_COLLECTION_NAME = "all_data_20251021"

# --- 2. LÓGICA PURA DE LA MÉTRICA ---

# Mapeo de claves de nivel educativo a un rango numérico para ordenar
# Basado en el JSON de ejemplo (BCH) y los archivos de métricas (1_3_01)
# NOTA: Estas claves son suposiciones y puede que necesites añadir más
LEVEL_RANK_MAP = {
    "PRI": 1,  # Primaria
    "SEC": 2,  # Secundaria
    "BCH": 3,  # Bachillerato
    "CTC": 3,  # Carrera Técnica o Comercial
    "LIC": 4,  # Licenciatura
    "ESP": 5,  # Especialidad
    "MTR": 6,  # Maestría
    "DOC": 7   # Doctorado
}

def parse_date(date_str: str) -> datetime:
    """
    Convierte un string YYYY-MM-DD a un objeto datetime.
    Devuelve None si el formato es incorrecto.
    """
    if not isinstance(date_str, str) or len(date_str) != 10:
        return None
    try:
        return datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        return None

def validar_congruencia_fechas_escolaridad(escolaridad_array: list) -> str:
    """
    Valida que las fechas de obtención de estudios sean cronológicas
    según el nivel de estudios.
    """
    
    # 1. Validar entrada
    if not isinstance(escolaridad_array, list) or len(escolaridad_array) < 2:
        # No hay nada que comparar si no hay al menos 2 estudios
        return "N/A"

    niveles_con_fecha = []
    
    # 2. Extraer y limpiar datos
    for estudio in escolaridad_array:
        if not isinstance(estudio, dict):
            continue # Ignorar si el item no es un objeto
            
        nivel_clave = estudio.get("nivel", {}).get("clave")
        fecha_str = estudio.get("fechaObtencion")
        
        # Solo procesamos estudios con nivel y fecha válidos
        if nivel_clave and fecha_str:
            rank = LEVEL_RANK_MAP.get(nivel_clave)
            fecha = parse_date(fecha_str)
            
            # Solo añadimos si pudimos rankear el nivel y parsear la fecha
            if rank and fecha:
                niveles_con_fecha.append((rank, fecha))

    # 3. Validar si tenemos suficientes datos para comparar
    if len(niveles_con_fecha) < 2:
        return "N/A" # No hay suficientes estudios con fechas válidas para comparar

    # 4. Ordenar la lista por el rango (nivel educativo)
    niveles_con_fecha.sort(key=lambda x: x[0])

    # 5. Comparar fechas cronológicamente
    try:
        for i in range(1, len(niveles_con_fecha)):
            rank_anterior, fecha_anterior = niveles_con_fecha[i-1]
            rank_actual, fecha_actual = niveles_con_fecha[i]
            
            # Si los niveles son iguales, no comparamos (ej. 2 licenciaturas)
            if rank_actual == rank_anterior:
                continue
                
            # Si el nivel actual es superior, su fecha debe ser posterior o igual
            if fecha_actual < fecha_anterior:
                return "NO_CUMPLE" # Error: Licenciatura antes que Bachillerato
                
    except Exception:
        return "SIN_DATO" # Error inesperado durante la comparación

    # Si el bucle termina sin problemas, es congruente
    return "CUMPLE"


# --- 3. EL FLUJO DEL WORKER (PROCESAMIENTO POR LOTES) ---
def procesar_todas_las_declaraciones_fechas_escolaridad():
    """
    Procesa TODAS las declaraciones para la Métrica 1.2.01
    """
    client = None
    # Contadores
    total_documentos = 0
    cumple = 0
    no_cumple = 0
    sin_dato = 0
    na = 0 # No Aplica
    
    try:
        # --- A. CONECTAR A LA BASE DE DATOS ---
        print(f"\nConectando a MongoDB en {DB_NAME}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        client.admin.command('ping') 
        db = client[DB_NAME]
        source_collection = db[SOURCE_COLLECTION_NAME]
        target_collection = db[METRICS_COLLECTION_NAME]
        
        print("¡Conexión exitosa!")
        print(f"--- INICIANDO PROCESAMIENTO (Métrica 1.2.01 - Fechas Escolaridad) ---")

        # Proyección optimizada
        proyeccion = {
            "_id": 1,
            "id": 1,
            "declaracion.situacionPatrimonial.datosCurricularesDeclarante.escolaridad": 1 
        }
        
        for doc in source_collection.find({}, proyeccion):
            total_documentos += 1
            original_id = doc["_id"]
            string_id = doc.get("id", "N/D")
            
            escolaridad_data = None
            resultado_metrica = "SIN_DATO" # Valor por defecto
            
            try:
                # Navegación segura
                escolaridad_data = doc.get("declaracion", {}).get("situacionPatrimonial", {}).get("datosCurricularesDeclarante", {}).get("escolaridad")
                resultado_metrica = validar_congruencia_fechas_escolaridad(escolaridad_data)

            except AttributeError:
                resultado_metrica = "SIN_DATO" # Ocurrió un error leyendo la estructura
            
            # Actualizamos contadores
            if resultado_metrica == "CUMPLE": cumple += 1
            elif resultado_metrica == "NO_CUMPLE": no_cumple += 1
            elif resultado_metrica == "N/A": na += 1
            else: sin_dato += 1 # SIN_DATO

            # --- CONSERVAMOS TU SALIDA DE CONSOLA ---
            print(f"\nProcesando Documento ID: {original_id} (id: {string_id})")
            print("--- RESULTADO DEL ANÁLISIS ---")
            # Imprimimos cuántos estudios tiene para referencia
            print(f"  > Estudios encontrados: {len(escolaridad_data) if isinstance(escolaridad_data, list) else 0}")
            print(f"  > Resultado de la Métrica: {resultado_metrica}")
            print("---------------------------------")

            # --- E. GUARDAR RESULTADO EN LA COLECCIÓN 'metricas' ---
            filtro = { "_id": original_id }
            actualizacion = { "$set": { METRIC_ID: resultado_metrica } }
            target_collection.update_one(filtro, actualizacion, upsert=True)
            print(f"¡Resultado guardado/actualizado en 'metricas' para el ID: {original_id}!")

        print("\n--- PROCESAMIENTO POR LOTES FINALIZADO ---")
        print("Resumen (Métrica 1.2.01):")
        print(f"  > Documentos Totales Procesados: {total_documentos}")
        print(f"  > Métrica 'CUMPLE': {cumple}")
        print(f"  > Métrica 'NO_CUMPLE': {no_cumple}")
        print(f"  > Métrica 'N/A' (1 o 0 estudios): {na}")
        print(f"  > Métrica 'SIN_DATO': {sin_dato}")

    except ConnectionFailure: print("Error: No se pudo conectar a la base de datos.")
    except OperationFailure as e: print(f"Error en la operación de la base de datos: {e.details}")
    except Exception as e:
        print(f"Ocurrió un error inesperado: {e}")
        traceback.print_exc()
    finally:
        if client: client.close(); print("Conexión cerrada.")

# --- 4. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    procesar_todas_las_declaraciones_fechas_escolaridad()


Conectando a MongoDB en sistema1...
¡Conexión exitosa!
--- INICIANDO PROCESAMIENTO (Métrica 1.2.01 - Fechas Escolaridad) ---

Procesando Documento ID: 68f81b8800535f910a29f694 (id: 194916)
--- RESULTADO DEL ANÁLISIS ---
  > Estudios encontrados: 0
  > Resultado de la Métrica: N/A
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f694!

Procesando Documento ID: 68f81b8800535f910a29f695 (id: 59837)
--- RESULTADO DEL ANÁLISIS ---
  > Estudios encontrados: 3
  > Resultado de la Métrica: CUMPLE
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f695!

Procesando Documento ID: 68f81b8800535f910a29f696 (id: 31074)
--- RESULTADO DEL ANÁLISIS ---
  > Estudios encontrados: 1
  > Resultado de la Métrica: N/A
---------------------------------
¡Resultado guardado/actualizado en 'metricas' para el ID: 68f81b8800535f910a29f696!

Procesando Documento ID: 68f81b8800535f910a29f697 

KeyboardInterrupt: 